In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
roads_df = gpd.read_file('../data/egypt/alex_roads.geojson').to_crs(epsg=4326)

In [3]:
import uuid
import random
from datetime import datetime, timedelta

index = roads_df.index

def generate_sample_crack_data(num_records=10):
    labels = ['crack', 'pothole', 'wear', 'none']
    base_time = datetime.now()
    sample_data = []

    for _ in range(num_records):
        road_index = random.choice(index)
        timestamp = base_time - timedelta(seconds=random.randint(0, 3600))
        label = random.choice(labels)
        confidence = round(random.uniform(0.6, 0.99), 2)
        image = f'image_{uuid.uuid4().hex[:8]}.jpg'  # Simulated image filename
        lon = round(random.uniform(-180, 180), 6)
        lat = round(random.uniform(-90, 90), 6)
        x1 = round(random.uniform(0, 1), 4)
        y1 = round(random.uniform(0, 1), 4)
        x2 = round(min(x1 + random.uniform(0, 0.5), 1), 4)
        y2 = round(min(y1 + random.uniform(0, 0.5), 1), 4)

        sample_data.append({
            'id': str(uuid.uuid4()),
            'road_index': road_index,
            'timestamp': timestamp,
            'label': label,
            'confidence': confidence,
            'image': image,
            'lon': lon,
            'lat': lat,
            'x1': x1,
            'y1': y1,
            'x2': x2,
            'y2': y2
        })

    return sample_data

# Example usage:
if __name__ == "__main__":
    data = generate_sample_crack_data(10000)
    df = pd.DataFrame(data)

In [4]:
roads_df['road_index'] = roads_df.index
joined = roads_df.merge(df, how='inner', left_on='road_index', right_on='road_index')

In [5]:
joined.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,timestamp,label,confidence,image,lon,lat,x1,y1,x2,y2
0,10867227,5115,tertiary,شارع اسماعيل سرى,None,F,0,0,F,F,...,2025-09-14 23:19:19.470906,pothole,0.61,image_7e3752fd.jpg,140.441737,-58.930204,0.4012,0.5812,0.4632,1.0000
1,24318273,5114,secondary,شارع عمر طوسون,None,F,0,0,F,F,...,2025-09-15 00:03:48.470906,none,0.61,image_7cb4b369.jpg,-74.903081,-69.829199,0.8843,0.9487,1.0000,1.0000
2,24581717,5131,motorway_link,None,None,F,0,1,T,F,...,2025-09-14 23:29:36.470906,pothole,0.75,image_8788a975.jpg,142.571365,58.876475,0.7131,0.3043,0.8356,0.4263
3,25188567,5122,residential,شارع قلعه قايتباى,None,F,0,0,F,F,...,2025-09-15 00:06:01.470906,crack,0.93,image_76a5bf47.jpg,-112.512651,36.819815,0.0726,0.2676,0.3644,0.7278
4,25190790,5131,motorway_link,None,27,F,0,0,F,F,...,2025-09-15 00:09:32.470906,pothole,0.82,image_201948e5.jpg,161.475809,-21.938533,0.5703,0.1291,0.6586,0.6191


In [6]:
joined.to_file("../data/egypt/sample.geojson", driver='GeoJSON')